In [1]:
import os
import zipfile
import glob

import pandas as pd
from Bio import SeqIO

In [2]:
# fasta = "ZMays/fasta"
# tes = "ZMays/tes"
# groups = "ZMays/Agrup"

fasta = "ZMays/fasta"
tes = "ZMays/tes"
groups = "ZMays/Agrup"

In [3]:
# Navigate to the directory containing the zip files
os.chdir(fasta)

# Iterate through the zip files
for i in range(1, 11):
  zip_filename = f"chromosome{i}.zip"

  #Check if the zip file exists
  if os.path.exists(zip_filename):
    try:
      with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall()
        print(f"Extração bem-sucedida {zip_filename}")
    except zipfile.BadZipFile:
      print(f"Erro: {zip_filename} não é um arquivo zip.")
    except Exception as e:
      print(f"Ocorreu um erro durante a extração {zip_filename}: {e}")
  else:
    print(f"Warning: {zip_filename} não encontrado.")

# Use glob to find all .zip files
zip_files = glob.glob("*.zip")

# Iterate through the found zip files
for zip_filename in zip_files:
    try:
        os.remove(zip_filename)
        print(f"Removido com sucesso {zip_filename}")
    except OSError as e:
        print(f"Erro: {zip_filename}: {e}")

In [4]:
ls

 O volume na unidade C n�o tem nome.
 O N�mero de S�rie do Volume � C6A6-CF4A

 Pasta de c:\Users\Vin�cius\Documents\USP\ZeaMays-TEsClassification\ZMays\fasta

29/06/2025  22:33    <DIR>          .
28/06/2025  15:17    <DIR>          ..
11/05/2025  17:08           144.446 chloroplast.fasta
29/06/2025  22:33       312.859.019 chromosome1.fasta
29/06/2025  22:33       154.613.105 chromosome10.fasta
29/06/2025  22:33       247.156.350 chromosome2.fasta
29/06/2025  22:33       241.418.105 chromosome3.fasta
29/06/2025  22:33       253.906.694 chromosome4.fasta
29/06/2025  22:33       229.587.154 chromosome5.fasta
29/06/2025  22:33       183.948.136 chromosome6.fasta
29/06/2025  22:33       188.463.414 chromosome7.fasta
29/06/2025  22:33       185.017.161 chromosome8.fasta
29/06/2025  22:33       165.333.468 chromosome9.fasta
11/05/2025  17:08           585.971 mitochondrion.fasta
              12 arquivo(s)  2.163.033.023 bytes
               2 pasta(s)    6.215.254.016 bytes dispon�veis


In [5]:
cd C:\Users\Vinícius\Documents\USP\ZeaMays-TEsClassification

C:\Users\Vinícius\Documents\USP\ZeaMays-TEsClassification


c:\Users\Vinícius\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
file_path = (tes + "/TEAnnotationFinal_Helitron.gff3")

with open(file_path, "r") as file:
    # Use list comprehension para pegar as primeiras 10 linhas
    for i, line in enumerate(file):
        print(line.strip())  # strip() remove espaços extras e quebras de linha duplicadas
        if i == 9:  # Exibe apenas as primeiras 10 linhas
            break

5	APTEdb	Class II subclass 2/Helitron/Helitron	28903351	28903636	.	-	.	TE-Score=0.285;Software=RepeatModeler;Length=285bps
3	APTEdb	Class II subclass 2/Helitron/Helitron	2184634	2185235	.	-	.	TE-Score=0.428;Software=RepeatMasker;Length=601bps
7	APTEdb	Class II subclass 2/Helitron/Helitron	82856122	82857584	.	+	.	TE-Score=0.428;Software=RepeatModeler;Length=1462bps
1	APTEdb	Class II subclass 2/Helitron/Helitron	239302471	239302834	.	+	.	TE-Score=0.285;Software=RepeatModeler;Length=363bps
10	APTEdb	Class II subclass 2/Helitron/Helitron	5646459	5648982	.	-	.	TE-Score=0.571;Software=HelitronScanner;Length=2523bps
1	APTEdb	Class II subclass 2/Helitron/Helitron	185134318	185151509	.	-	.	TE-Score=0.714;Software=HelitronScanner;Length=17191bps
2	APTEdb	Class II subclass 2/Helitron/Helitron	240141757	240157943	.	-	.	TE-Score=0.714;Software=HelitronScanner;Length=16186bps
8	APTEdb	Class II subclass 2/Helitron/Helitron	2261318	2261604	.	+	.	TE-Score=0.142;Software=RepeatModeler;Length=286bps
2	AP

In [7]:
def add_gff3_to_dataframe(df, file_path):
    """
    Lê um arquivo GFF3, extrai os dados e adiciona ao DataFrame existente.

    Parâmetros:
        df (pd.DataFrame): DataFrame existente para o qual os dados serão adicionados.
        file_path (str): Caminho do arquivo GFF3.

    Retorna:
        pd.DataFrame: DataFrame atualizado com os novos dados.
    """
    # Definir os nomes das colunas do arquivo GFF3
    column_names = ["Chr", "SourceAnnotation", "COS", "Start", "End", "Score", "Strand", "Phase", "Attributes"]

    # Lista para armazenar os dados do novo arquivo
    data = []

    try:
        with open(file_path, "r") as file:
            for line in file:
                if not line.startswith("#"):  # Ignorar linhas de comentários
                    parts = line.strip().split("\t")
                    if len(parts) == len(column_names):  # Verificar o número correto de colunas
                        data.append(parts)
                    else:
                        print(f"Warning: Skipping line with incorrect number of columns: {line.strip()}")

        # Criar um DataFrame com os dados lidos
        new_df = pd.DataFrame(data, columns=column_names)

        # Adicionar os novos dados ao DataFrame existente
        updated_df = pd.concat([df, new_df], ignore_index=True)
        return updated_df

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

    return df  # Retorna o DataFrame original caso haja um erro


In [8]:
# helitron = "/content/ZeaMays-TEsClassification/ZMays/tes/TEAnnotationFinal_Helitron.gff3"
# line = "/content/ZeaMays-TEsClassification/ZMays/tes/TEAnnotationFinal_LINE.gff3"
# ltr = "/content/ZeaMays-TEsClassification/ZMays/tes/TEAnnotationFinal_LTR.gff3"
# mite = "/content/ZeaMays-TEsClassification/ZMays/tes/TEAnnotationFinal_MITE.gff3"
# sine = "/content/ZeaMays-TEsClassification/ZMays/tes/TEAnnotationFinal_SINE.gff3"
# tir = "/content/ZeaMays-TEsClassification/ZMays/tes/TEAnnotationFinal_TIR.gff3"

helitron = "ZMays/tes/TEAnnotationFinal_Helitron.gff3"
line = "ZMays/tes/TEAnnotationFinal_LINE.gff3"
ltr = "ZMays/tes/TEAnnotationFinal_LTR.gff3"
mite = "ZMays/tes/TEAnnotationFinal_MITE.gff3"
sine = "ZMays/tes/TEAnnotationFinal_SINE.gff3"
tir = "ZMays/tes/TEAnnotationFinal_TIR.gff3"

In [9]:
df = ""
df = pd.DataFrame(columns=["Chr", "SourceAnnotation", "COS", "Start", "End", "Score", "Strand", "Phase", "Attributes"])

df = add_gff3_to_dataframe(df, helitron)
df = add_gff3_to_dataframe(df, line)
df = add_gff3_to_dataframe(df, ltr)
df = add_gff3_to_dataframe(df, mite)
df = add_gff3_to_dataframe(df, sine)
df = add_gff3_to_dataframe(df, tir)

print(df.head())

  Chr SourceAnnotation                                    COS      Start  \
0   5           APTEdb  Class II subclass 2/Helitron/Helitron   28903351   
1   3           APTEdb  Class II subclass 2/Helitron/Helitron    2184634   
2   7           APTEdb  Class II subclass 2/Helitron/Helitron   82856122   
3   1           APTEdb  Class II subclass 2/Helitron/Helitron  239302471   
4  10           APTEdb  Class II subclass 2/Helitron/Helitron    5646459   

         End Score Strand Phase  \
0   28903636     .      -     .   
1    2185235     .      -     .   
2   82857584     .      +     .   
3  239302834     .      +     .   
4    5648982     .      -     .   

                                          Attributes  
0  TE-Score=0.285;Software=RepeatModeler;Length=2...  
1  TE-Score=0.428;Software=RepeatMasker;Length=60...  
2  TE-Score=0.428;Software=RepeatModeler;Length=1...  
3  TE-Score=0.285;Software=RepeatModeler;Length=3...  
4  TE-Score=0.571;Software=HelitronScanner;Length...  


In [10]:
#df = df[df['Strand'] == '+']
df.reset_index(drop=True, inplace=True)

print(df.head())
print(f"\n\nNúmero de entradas: {len(df)}")

  Chr SourceAnnotation                                    COS      Start  \
0   5           APTEdb  Class II subclass 2/Helitron/Helitron   28903351   
1   3           APTEdb  Class II subclass 2/Helitron/Helitron    2184634   
2   7           APTEdb  Class II subclass 2/Helitron/Helitron   82856122   
3   1           APTEdb  Class II subclass 2/Helitron/Helitron  239302471   
4  10           APTEdb  Class II subclass 2/Helitron/Helitron    5646459   

         End Score Strand Phase  \
0   28903636     .      -     .   
1    2185235     .      -     .   
2   82857584     .      +     .   
3  239302834     .      +     .   
4    5648982     .      -     .   

                                          Attributes  
0  TE-Score=0.285;Software=RepeatModeler;Length=2...  
1  TE-Score=0.428;Software=RepeatMasker;Length=60...  
2  TE-Score=0.428;Software=RepeatModeler;Length=1...  
3  TE-Score=0.285;Software=RepeatModeler;Length=3...  
4  TE-Score=0.571;Software=HelitronScanner;Length...  




In [11]:
# Ordernar o DataFrame
df['Start'] = df['Start'].astype(int)
df['End'] = df['End'].astype(int)
df_sorted = df.sort_values(by=['Chr', 'Start'])

# Reset the index of the sorted DataFrame
df_sorted.reset_index(drop=True, inplace=True)

print(df_sorted.head())
print(f"\n\nNúmero de entradas: {len(df_sorted)}")

  Chr SourceAnnotation                                    COS  Start    End  \
0   1           APTEdb  Class II subclass 2/Helitron/Helitron  27169  31243   
1   1           APTEdb                    Class I/LTR/Unknown  31390  31484   
2   1           APTEdb                      Class I/LTR/Gypsy  31405  31493   
3   1           APTEdb                    Class I/LTR/Unknown  31573  31667   
4   1           APTEdb                      Class I/LTR/Gypsy  31596  31679   

  Score Strand Phase                                         Attributes  
0     .      +     .  TE-Score=0.714;Software=HelitronScanner;Length...  
1     .      -   287  TE-Score=0.285;Software=LTRRetriever;Length=94bps  
2     .      +     .  TE-Score=0.428;Software=RepeatModeler;Length=8...  
3     .      -   281  TE-Score=0.142;Software=LTRRetriever;Length=94bps  
4     .      +     .  TE-Score=0.142;Software=RepeatModeler;Length=8...  


Número de entradas: 678138


In [12]:
# Lista dos tipos permitidos
types = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]

# Filtrar o DataFrame para manter apenas os tipos permitidos
df = df[df["Chr"].isin(types)]
df.reset_index(drop=True, inplace=True)

# Sumarização
summary = df["Chr"].value_counts()

for index, value in summary.items():
    print(f"{index}: {value}")

print(f"\n\nNúmero de entradas: {len(df)}")

4: 100427
1: 97069
2: 74309
3: 72265
5: 68571
8: 55876
7: 55250
6: 51638
9: 49783
10: 46214


Número de entradas: 671402


In [13]:
df.head()

,Chr,SourceAnnotation,COS,Start,End,Score,Strand,Phase,Attributes
0,5,APTEdb,Class II subclass 2/Helitron/Helitron,28903351,28903636,.,-,.,TE-Score=0.285;Software=RepeatModeler;Length=2...
1,3,APTEdb,Class II subclass 2/Helitron/Helitron,2184634,2185235,.,-,.,TE-Score=0.428;Software=RepeatMasker;Length=60...
2,7,APTEdb,Class II subclass 2/Helitron/Helitron,82856122,82857584,.,+,.,TE-Score=0.428;Software=RepeatModeler;Length=1...
3,1,APTEdb,Class II subclass 2/Helitron/Helitron,239302471,239302834,.,+,.,TE-Score=0.285;Software=RepeatModeler;Length=3...
4,10,APTEdb,Class II subclass 2/Helitron/Helitron,5646459,5648982,.,-,.,TE-Score=0.571;Software=HelitronScanner;Length...


In [14]:
dfs_metrics = []

i = 0
for f in glob.glob(groups + "/*.csv"):
  df_metric = pd.read_csv(f)
  df_metric = df_metric.sort_values(by='nameseq')

  columns = [f"{f.replace('ZMays/Agrup/', '').replace('.csv', '')}_{column}" for column in df_metric.columns]
  if i != 0:
    df_metric.drop(columns=['nameseq'], inplace=True)
    columns.remove(columns[0])
  else:
    columns[0] = 'nameseq'

  df_metric.columns = columns

  df_metric.reset_index(drop=True, inplace=True)
  dfs_metrics.append(df_metric)
  i += 1

df_metrics = pd.concat(dfs_metrics, axis=1)
df_metrics.head()

,nameseq,ZMays/Agrup\anf_fourier_average,ZMays/Agrup\anf_fourier_median,ZMays/Agrup\anf_fourier_maximum,ZMays/Agrup\anf_fourier_minimum,ZMays/Agrup\anf_fourier_peak,ZMays/Agrup\anf_fourier_none_levated_peak,ZMays/Agrup\anf_fourier_sample_standard_deviation,ZMays/Agrup\anf_fourier_population_standard_deviation,ZMays/Agrup\anf_fourier_percentile15,...,ZMays/Agrup\orf_minimum_ORF_length,ZMays/Agrup\orf_std_ORF_length,ZMays/Agrup\orf_average_ORF_length,ZMays/Agrup\orf_cv_ORF_length,ZMays/Agrup\orf_maximum_GC_content_ORF,ZMays/Agrup\orf_minimum_GC_content_ORF,ZMays/Agrup\orf_std_GC_content_ORF,ZMays/Agrup\orf_average_GC_content_ORF,ZMays/Agrup\orf_cv_GC_content_ORF,ZMays/Agrup\orf_class
0,10_104431_104500,8.248204,0.495401,481.523442,0.001764,2.828899,18.355640,57.003340,57.414922,0.166519,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
1,10_104817_104874,10.103667,2.135769,390.545718,0.015395,1.913497,10.105725,50.544025,50.985466,0.400310,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
2,10_104957_105096,13.674118,1.140519,1674.002251,0.014161,3.412774,32.829516,140.838334,141.344039,0.170290,...,15.0,0.000000,15.000000,0.000000,26.666667,26.666667,0.000000,26.666667,0.000000,NaN
3,10_106081_106190,13.922727,1.439795,1292.065720,0.055156,2.633584,22.574048,122.443130,123.003513,0.403268,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
4,10_106081_108136,171.914850,10.534463,322090.374942,0.038343,3.986470,183.905772,7101.353795,7103.081408,2.586421,...,9.0,21.175513,32.538462,0.650784,44.444444,16.666667,8.278575,32.995014,0.250904,NaN


In [15]:
print(df_metrics['nameseq'].astype(str))

0        10_104431_104500
1        10_104817_104874
2        10_104957_105096
3        10_106081_106190
4        10_106081_108136
               ...       
99089                 nan
99090                 nan
99091                 nan
99092                 nan
99093                 nan
Name: nameseq, Length: 99094, dtype: object


In [16]:
mask = df_metrics['nameseq'].apply(lambda x: isinstance(x, str))
df_filtrado = df_metrics.loc[mask]

In [17]:
mascara = df_metrics['nameseq'].map(type) == str

# 2. Aplica a máscara para filtrar o DataFrame
df_metrics_filtrado = df_metrics[mascara]

print("DataFrame original:")
print(df_metrics)
print("\nDataFrame filtrado (apenas strings em 'nameseq'):")
print(df_metrics_filtrado)

DataFrame original:
                nameseq  ZMays/Agrup\anf_fourier_average  \
0      10_104431_104500                         8.248204   
1      10_104817_104874                        10.103667   
2      10_104957_105096                        13.674118   
3      10_106081_106190                        13.922727   
4      10_106081_108136                       171.914850   
...                 ...                              ...   
99089               NaN                              NaN   
99090               NaN                              NaN   
99091               NaN                              NaN   
99092               NaN                              NaN   
99093               NaN                              NaN   

       ZMays/Agrup\anf_fourier_median  ZMays/Agrup\anf_fourier_maximum  \
0                            0.495401                       481.523442   
1                            2.135769                       390.545718   
2                            1.140519

In [18]:
print(df)

       Chr SourceAnnotation                                    COS      Start  \
0        5           APTEdb  Class II subclass 2/Helitron/Helitron   28903351   
1        3           APTEdb  Class II subclass 2/Helitron/Helitron    2184634   
2        7           APTEdb  Class II subclass 2/Helitron/Helitron   82856122   
3        1           APTEdb  Class II subclass 2/Helitron/Helitron  239302471   
4       10           APTEdb  Class II subclass 2/Helitron/Helitron    5646459   
...     ..              ...                                    ...        ...   
671397   1           APTEdb         Class II subclass 1/TIR/Ginger  223982226   
671398   4           APTEdb         Class II subclass 1/TIR/Ginger  118840026   
671399   4           APTEdb         Class II subclass 1/TIR/Ginger    7068255   
671400   4           APTEdb         Class II subclass 1/TIR/Ginger   13348879   
671401   1           APTEdb         Class II subclass 1/TIR/Ginger  253818258   

              End Score Str

In [19]:
print(df_metrics_filtrado)

                   nameseq  ZMays/Agrup\anf_fourier_average  \
0         10_104431_104500                         8.248204   
1         10_104817_104874                        10.103667   
2         10_104957_105096                        13.674118   
3         10_106081_106190                        13.922727   
4         10_106081_108136                       171.914850   
...                    ...                              ...   
98981    9_9988349_9989443                        81.082109   
98982  9_99909901_99910036                        17.399278   
98983  9_99924376_99930419                       416.610410   
98984  9_99973193_99973261                         9.911594   
98985    9_9997996_9999090                        84.597975   

       ZMays/Agrup\anf_fourier_median  ZMays/Agrup\anf_fourier_maximum  \
0                            0.495401                     4.815234e+02   
1                            2.135769                     3.905457e+02   
2                    

In [20]:
import os
import pandas as pd
from typing import List, Optional

def juntar_csvs_em_dataframe(lista_arquivos_csv: List[str]) -> Optional[pd.DataFrame]:
    """
    Lê uma lista de arquivos CSV, concatena-os e retorna um único DataFrame.

    Parâmetros:
        lista_arquivos_csv (List[str]): Uma lista contendo os caminhos para os arquivos CSV.

    Retorna:
        pd.DataFrame: DataFrame concatenado com todos os dados, ou None se ocorrer um erro.
    """
    lista_dfs = []
    try:
        for arquivo_csv in lista_arquivos_csv:
            print(f"Lendo o arquivo para junção: '{arquivo_csv}'...")
            df_temp = pd.read_csv(arquivo_csv)
            print(f"Dimensões de '{os.path.basename(arquivo_csv)}': {df_temp.shape[0]} linhas, {df_temp.shape[1]} colunas")
            lista_dfs.append(df_temp)

        if not lista_dfs:
            print("Erro: Nenhum arquivo foi lido com sucesso.")
            return None

        df_concatenado = pd.concat(lista_dfs, ignore_index=True)
        print(f"\\nDimensões do DataFrame final concatenado: {df_concatenado.shape[0]} linhas, {df_concatenado.shape[1]} colunas")
        print(f"{len(lista_arquivos_csv)} arquivos CSV juntados em um DataFrame com sucesso!")
        return df_concatenado

    except FileNotFoundError as e:
        print(f"Erro: O arquivo {e.filename} não foi encontrado.")
        return None
    except Exception as e:
        print(f"Ocorreu um erro ao juntar os arquivos CSV: {e}")
        return None

def juntar_dataframes_inner(df1: pd.DataFrame, df2: pd.DataFrame) -> Optional[pd.DataFrame]:
    """
    Realiza a junção interna (inner join) de dois DataFrames na coluna 'nameseq'.
    """
    if 'nameseq' not in df1.columns or 'nameseq' not in df2.columns:
        print("Erro: A coluna 'nameseq' não está presente em ambos os DataFrames.")
        return None
    
    df_juntado = pd.merge(df1, df2, on='nameseq', how='inner')
    print(f"DataFrames juntados com 'inner join' na coluna 'nameseq'. Dimensões do resultado: {df_juntado.shape}")
    return df_juntado

In [21]:

arquivos_kmer = [
    'ZMays/Agrup/k-mer/divididos/k-mer_agrupado_parte_1_de_5.zip',
    'ZMays/Agrup/k-mer/divididos/k-mer_agrupado_parte_2_de_5.zip',
    'ZMays/Agrup/k-mer/divididos/k-mer_agrupado_parte_3_de_5.zip',
    'ZMays/Agrup/k-mer/divididos/k-mer_agrupado_parte_4_de_5.zip',
    'ZMays/Agrup/k-mer/divididos/k-mer_agrupado_parte_5_de_5.zip'
]

# Chama a função para concatenar os 5 arquivos k-mer em um único DataFrame
df_kmer = juntar_csvs_em_dataframe(arquivos_kmer)

# Prossegue com a junção interna (inner join) se o df_kmer foi criado com sucesso
if df_kmer is not None:
    df_all_metrics = juntar_dataframes_inner(df_metrics_filtrado, df_kmer)
    if df_all_metrics is not None:
        print(df_all_metrics.head())

Lendo o arquivo para junção: 'ZMays/Agrup/k-mer/divididos/k-mer_agrupado_parte_1_de_5.zip'...
Dimensões de 'k-mer_agrupado_parte_1_de_5.zip': 19798 linhas, 5462 colunas
Lendo o arquivo para junção: 'ZMays/Agrup/k-mer/divididos/k-mer_agrupado_parte_2_de_5.zip'...
Dimensões de 'k-mer_agrupado_parte_2_de_5.zip': 19798 linhas, 5462 colunas
Lendo o arquivo para junção: 'ZMays/Agrup/k-mer/divididos/k-mer_agrupado_parte_3_de_5.zip'...
Dimensões de 'k-mer_agrupado_parte_3_de_5.zip': 19797 linhas, 5462 colunas
Lendo o arquivo para junção: 'ZMays/Agrup/k-mer/divididos/k-mer_agrupado_parte_4_de_5.zip'...
Dimensões de 'k-mer_agrupado_parte_4_de_5.zip': 19797 linhas, 5462 colunas
Lendo o arquivo para junção: 'ZMays/Agrup/k-mer/divididos/k-mer_agrupado_parte_5_de_5.zip'...
Dimensões de 'k-mer_agrupado_parte_5_de_5.zip': 19797 linhas, 5462 colunas
\nDimensões do DataFrame final concatenado: 98987 linhas, 5462 colunas
5 arquivos CSV juntados em um DataFrame com sucesso!
DataFrames juntados com 'inner 

In [22]:
print(df_all_metrics.shape)

(98986, 5771)


In [23]:
import pandas as pd
import numpy as np # Para pd.NA ou np.nan, se necessário

def join_metrics_inner(df_main, df_metrics_values):
    
    df_metrics = df_metrics_values.copy()

    # Dividir 'nameseq' em colunas chave
    try:
        
        split_data = df_metrics['nameseq'].str.split('_', expand=True, n=2)
        
        if split_data.shape[1] < 3:
            raise ValueError(
                "A coluna 'nameseq' em df_metrics_values não pôde ser dividida em 3 partes (Chr, Start, End) consistentemente."
            )
        
        df_metrics['_Chr_key_metric'] = split_data[0].astype(str)
        df_metrics['_Start_key_metric'] = split_data[1].astype(str)
        df_metrics['_End_key_metric'] = split_data[2].astype(str)
        
    except Exception as e:
        
        print(f"Erro ao processar a coluna 'nameseq': {e}")
        print("Verifique o formato dos dados em 'nameseq'. Esperado: 'Chr_Start_End'.")
       
        raise

    # Identificar as colunas de métricas que serão transferidas/atualizadas
    metric_value_cols = [
        col for col in df_metrics.columns
        if col not in ['nameseq', '_Chr_key_metric', '_Start_key_metric', '_End_key_metric']
    ]
    
    # Selecionar apenas as colunas chave e de métricas para a junção
    df_metrics_to_join = df_metrics[
        ['_Chr_key_metric', '_Start_key_metric', '_End_key_metric'] + metric_value_cols
    ]

    df_left = df_main.copy()
    
    # Adicionar colunas chave temporárias em df_left, convertendo para string para correspondência
    df_left['_Chr_key_df'] = df_left['Chr'].astype(str)
    df_left['_Start_key_df'] = df_left['Start'].astype(str)
    df_left['_End_key_df'] = df_left['End'].astype(str)

    merged_df = pd.merge(
        df_left,
        df_metrics_to_join,
        left_on=['_Chr_key_df', '_Start_key_df', '_End_key_df'],
        right_on=['_Chr_key_metric', '_Start_key_metric', '_End_key_metric'],
        how='inner', # Especifica a junção interna
        suffixes=('', '_newmetric')
    )

    
    for metric_col_name in metric_value_cols:
        
        suffixed_metric_col_name = metric_col_name + '_newmetric'
        
        if suffixed_metric_col_name in merged_df.columns:
            
            merged_df[metric_col_name] = merged_df[suffixed_metric_col_name]
            merged_df.drop(columns=[suffixed_metric_col_name], inplace=True)
            
    cols_to_drop = [
        '_Chr_key_df', '_Start_key_df', '_End_key_df',
        '_Chr_key_metric', '_Start_key_metric', '_End_key_metric' # Remover se ainda existirem
    ]
    
    existing_cols_to_drop = [col for col in cols_to_drop if col in merged_df.columns]
    merged_df.drop(columns=existing_cols_to_drop, inplace=True, errors='ignore')
    
    return merged_df

In [24]:
df_resultado_juncao = join_metrics_inner(df, df_all_metrics)
print(df_resultado_juncao)

       Chr SourceAnnotation                                    COS      Start  \
0        5           APTEdb  Class II subclass 2/Helitron/Helitron   28903351   
1        3           APTEdb  Class II subclass 2/Helitron/Helitron    2184634   
2        7           APTEdb  Class II subclass 2/Helitron/Helitron   82856122   
3        1           APTEdb  Class II subclass 2/Helitron/Helitron  185134318   
4        8           APTEdb  Class II subclass 2/Helitron/Helitron    2261318   
...     ..              ...                                    ...        ...   
107566   4           APTEdb    Class II subclass 1/TIR/hAT-Charlie  233317063   
107567   6           APTEdb    Class II subclass 1/TIR/hAT-Charlie  164032515   
107568   4           APTEdb    Class II subclass 1/TIR/hAT-Charlie  200057025   
107569   2           APTEdb    Class II subclass 1/TIR/hAT-Charlie   13648304   
107570   4           APTEdb    Class II subclass 1/TIR/hAT-Charlie  233261821   

              End Score Str

In [25]:
df_met_seq = 'df_metricas_seq_final.csv'

try:
    df_resultado_juncao.to_csv(df_met_seq, index=False, encoding='utf-8')
    print(f"DataFrame salvo com sucesso como '{df_met_seq}'!")
except Exception as e:
    print(f"Ocorreu um erro ao salvar o DataFrame: {e}")

DataFrame salvo com sucesso como 'df_metricas_seq_final.csv'!
